In [2]:
import pandas as pd
import numpy as np
import csv

In [3]:
with open('./data/TSP-mat.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    data_list = list(reader)
    data = [row[1:] for row in data_list[1:]]

data = np.array(data, dtype=float)
print(np.shape(data))
print(data)

(107, 107)
[[  0.    5.    3.2 ... 124.5  81.5  41.4]
 [  5.    0.    4.9 ... 119.5  76.5  36.4]
 [  3.2   5.1   0.  ... 123.   79.7  34.3]
 ...
 [124.4 119.4 122.3 ...   0.   41.3 160.5]
 [ 81.8  76.8  80.9 ...  41.3   0.  116.4]
 [ 42.2  37.2  33.9 ... 160.5 116.4   0. ]]


## 验证两边之和大于第三边

In [38]:
res = 0;
for i in range(np.shape(data)[0]):
    for j in range(np.shape(data)[0]):
        for k in range(np.shape(data)[0]):
            if i != j and j != k and i != k:
                # if data[i][j] + data[j][k] < data[i][k]:
                sum_dis = data[i][j] + data[j][k]
                if (data[i][k] - sum_dis) / sum_dis > 0.1:
                    res += 1

print(res)
print(res / (np.shape(data)[0] * (np.shape(data)[0] - 1) * (np.shape(data)[0]) - 2))

9055
0.007461321432573715


不成立

## 使用ortools求解

In [4]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

load D:\miniconda3\envs\Callonetta\Lib\site-packages\ortools\.libs\zlib1.dll...
load D:\miniconda3\envs\Callonetta\Lib\site-packages\ortools\.libs\abseil_dll.dll...
load D:\miniconda3\envs\Callonetta\Lib\site-packages\ortools\.libs\utf8_validity.dll...
load D:\miniconda3\envs\Callonetta\Lib\site-packages\ortools\.libs\re2.dll...
load D:\miniconda3\envs\Callonetta\Lib\site-packages\ortools\.libs\libprotobuf.dll...
load D:\miniconda3\envs\Callonetta\Lib\site-packages\ortools\.libs\highs.dll...
load D:\miniconda3\envs\Callonetta\Lib\site-packages\ortools\.libs\ortools.dll...


In [4]:
def solve_tsp_ortools(tsp_matrix):
    num_nodes = len(tsp_matrix)
    manager = pywrapcp.RoutingIndexManager(num_nodes, 1, 0)
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return int(tsp_matrix[from_node][to_node] * 10)

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )

    # # 启用多线程 - 根据CPU核心数设置
    # import multiprocessing
    # num_cores = multiprocessing.cpu_count()
    # # search_parameters.number_of_threads = max(1, int(num_cores * 0.75))
    # search_parameters.local_search_metaheuristic_threads = max(1, int(num_cores * 0.75))

    # 其他性能优化参数
    search_parameters.time_limit.seconds = 600
    search_parameters.lns_time_limit.seconds = 60  # 大邻域搜索时间限制
    search_parameters.use_full_propagation = True  # 完全约束传播

    # 并行化策略
    search_parameters.local_search_operators.use_path_lns = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_inactive_lns = pywrapcp.BOOL_TRUE

    solution = routing.SolveWithParameters(search_parameters)
    if solution:
        index = routing.Start(0)
        tour = []
        while not routing.IsEnd(index):
            tour.append(manager.IndexToNode(index))
            index = solution.Value(routing.NextVar(index))
        tour.append(0)
        total_distance = solution.ObjectiveValue()
        return tour, total_distance
    else:
        return None, None

In [5]:
from ortools.sat.python import cp_model

def solve_tsp_ortools_new(tsp_matrix, time_limit_sec=600, num_workers=20):
    """
    用 CP-SAT 求解 TSP 并行版。

    参数:
        tsp_matrix     : 二维距离矩阵（浮点数或整数）。
        time_limit_sec : 搜索时间（秒）。
        num_workers    : 并行线程数。

    返回:
        tour           : 包含回到起点的结点序列列表。
        total_distance : 最优路程（原距离单位，浮点数）。
    """
    num_nodes = len(tsp_matrix)

    # 1. 建模型
    model = cp_model.CpModel()

    # 2. 定义二元决策变量 x[i,j]：是否从 i 走到 j
    x = {}
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i == j:
                continue
            x[(i, j)] = model.NewBoolVar(f'x_{i}_{j}')

    # 3. Circuit 约束：强制选出一条覆盖所有节点的回路
    arcs = []
    for (i, j), var in x.items():
        arcs.append((i, j, var))
    model.AddCircuit(arcs)

    # 4. 目标函数：最小化总距离
    terms = []
    for (i, j), var in x.items():
        # 把浮点距离放大 10 倍转成整数
        cost = int(tsp_matrix[i][j] * 10)
        terms.append(cost * var)
    model.Minimize(sum(terms))

    # 5. 调用 CP-SAT 求解器
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = time_limit_sec
    solver.parameters.num_search_workers = num_workers
    # solver.parameters.log_search_progress = True  # 如需日志可打开

    status = solver.Solve(model)

    # 6. 提取结果
    if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
        tour = [0]
        current = 0
        # 从 0 开始，按选中的 x[current,j] 一步步追踪
        for _ in range(num_nodes - 1):
            for j in range(num_nodes):
                if current != j and solver.Value(x[(current, j)]) == 1:
                    tour.append(j)
                    current = j
                    break
        tour.append(0)  # 回到起点

        # 解的 Objective 是放大 10 倍后的距离之和
        total_distance = solver.ObjectiveValue() / 10.0
        return tour, total_distance
    else:
        return None, None


# 示例调用
if __name__ == '__main__':
    # 简单示例矩阵
    matrix = np.random.randint(1, 100, size=(20, 20))
    np.fill_diagonal(matrix, 0)
    tour, dist = solve_tsp_ortools_new(matrix, time_limit_sec=30, num_workers=4)
    print('Tour:', tour)
    print('Distance:', dist)


Tour: [0, 5, 14, 13, 19, 12, 16, 3, 6, 8, 9, 11, 15, 4, 18, 7, 10, 17, 1, 2, 0]
Distance: 205.0


In [6]:
import multiprocessing
num_cores = multiprocessing.cpu_count()
print(num_cores)

32


In [7]:
tour, total_distance = solve_tsp_ortools_new(data)

if tour:
    print("Optimal tour:", tour)
    print("Total distance:", total_distance)
else:
    print("No solution found.")

Optimal tour: [0, 1, 5, 8, 105, 104, 97, 98, 95, 96, 94, 89, 90, 93, 74, 70, 91, 92, 76, 69, 68, 66, 67, 72, 73, 71, 75, 27, 18, 17, 24, 25, 20, 19, 28, 23, 21, 26, 22, 40, 10, 11, 30, 29, 41, 106, 34, 33, 35, 36, 37, 38, 39, 32, 31, 12, 6, 50, 15, 13, 14, 16, 65, 62, 61, 60, 53, 57, 58, 59, 82, 81, 80, 78, 79, 77, 85, 84, 88, 86, 83, 56, 87, 55, 54, 63, 64, 51, 46, 47, 49, 48, 42, 43, 45, 52, 44, 100, 99, 102, 101, 103, 9, 7, 3, 4, 2, 0]
Total distance: 6382.7


In [16]:
from ortools.sat.python import cp_model

def solve_two_traveler_tsp(
    tsp_matrix,
    origins,
    time_limit_sec=600,
    num_workers=8
):
    """
    用 CP-SAT 求解“两人旅行商”：
      - 两辆车从各自 origins[k] 出发并最终回到 origins[k]
      - 所有节点（包括 origins）在两条回路的并集上被访问，且恰好访问一次
      - 并行搜索 num_workers 线程，搜索总时限 time_limit_sec 秒

    返回:
      tours: [[...], [...]]  两辆车各自的访问序列，均以 origins[k] 开头、结束
      total_dist: float     两条回路距离之和
    """
    num_nodes = len(tsp_matrix)
    K = len(origins)
    model = cp_model.CpModel()

    # 1) 决策变量 x[k,i,j]：第 k 辆车是否从 i → j
    x = {}
    for k in range(K):
        for i in range(num_nodes):
            for j in range(num_nodes):
                if i != j:
                    x[k, i, j] = model.NewBoolVar(f"x[{k},{i},{j}]")

    # 2) 访问约束：除 origins 外，每个节点被恰好一次进入 & 退出
    for i in range(num_nodes):
        if i not in origins:
            # 进入总次数=1
            model.Add(
                sum(x[k, j, i] for k in range(K) for j in range(num_nodes) if j != i)
                == 1
            )
            # 退出总次数=1
            model.Add(
                sum(x[k, i, j] for k in range(K) for j in range(num_nodes) if j != i)
                == 1
            )

    # 3) origins 出发／回到约束
    for k, o in enumerate(origins):
        # 该车从 o 出发且仅出发一次
        model.Add(sum(x[k, o, j] for j in range(num_nodes) if j != o) == 1)
        # 该车回到 o 且仅回到一次
        model.Add(sum(x[k, i, o] for i in range(num_nodes) if i != o) == 1)
        # 其他车辆不得经过 o
        for k2 in range(K):
            if k2 != k:
                model.Add(sum(x[k2, o, j] for j in range(num_nodes) if j != o) == 0)
                model.Add(sum(x[k2, i, o] for i in range(num_nodes) if i != o) == 0)

    # 4) 流量守恒：非 origins 节点进入次数 = 退出次数
    for k in range(K):
        for i in range(num_nodes):
            if i not in origins:
                model.Add(
                    sum(x[k, j, i] for j in range(num_nodes) if j != i)
                    == sum(x[k, i, j] for j in range(num_nodes) if j != i)
                )

    # 5) MTZ 子回路消除（Miller–Tucker–Zemlin 变量）
    #    origins 的 u 定为 0，其它节点 u ∈ [1, num_nodes-1]
    u = {}
    for k in range(K):
        for i in range(num_nodes):
            if i in origins:
                u[k, i] = model.NewConstant(0)
            else:
                u[k, i] = model.NewIntVar(1, num_nodes - 1, f"u[{k},{i}]")
    #    对每条可能用到的弧 i→j 添加 MTZ 约束
    big_M = num_nodes
    for k in range(K):
        for i in range(num_nodes):
            for j in range(num_nodes):
                if i != j:
                    # u[i] + 1 ≤ u[j] + M*(1 − x[i,j])
                    model.Add(
                        u[k, i] + 1 <= u[k, j] + big_M * (1 - x[k, i, j])
                    )

    # 6) 目标：最小化两条回路总距离
    obj_terms = []
    for k in range(K):
        for i in range(num_nodes):
            for j in range(num_nodes):
                if i != j:
                    # 浮点放大 10 倍取整
                    cost = int(tsp_matrix[i][j] * 10)
                    obj_terms.append(cost * x[k, i, j])
    model.Minimize(sum(obj_terms))

    # 7) 求解器设置
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = time_limit_sec
    solver.parameters.num_search_workers = num_workers
    # solver.parameters.log_search_progress = True  # 如需调试时打开

    status = solver.Solve(model)
    if status not in (cp_model.OPTIMAL, cp_model.FEASIBLE):
        return None, None

    # 8) 提取回路
    tours = []
    for k, o in enumerate(origins):
        tour = [o]
        curr = o
        visited = {o}
        while True:
            nxt = None
            for j in range(num_nodes):
                if j != curr and solver.Value(x[k, curr, j]) == 1:
                    nxt = j
                    break
            if nxt is None or nxt == o:
                break
            tour.append(nxt)
            visited.add(nxt)
            curr = nxt
        tour.append(o)
        tours.append(tour)

    total_dist = solver.ObjectiveValue() / 10.0
    return tours, total_dist


# ------------ 测试示例 ------------
if __name__ == "__main__":
    matrix = [
        [0, 2.5, 9.0, 3.1, 7.2],
        [2.5, 0, 4.4, 8.5, 1.9],
        [9.0, 4.4, 0, 6.6, 5.5],
        [3.1, 8.5, 6.6, 0, 2.2],
        [7.2, 1.9, 5.5, 2.2, 0],
    ]
    origins = [0, 1]  # 旅行者 A 从节点 0 出发；B 从节点 1 出发
    tours, dist = solve_two_traveler_tsp(
        matrix, origins, time_limit_sec=60, num_workers=4
    )
    print("Tours:", tours)
    print("Total distance:", dist)


Tours: None
Total distance: None


In [27]:
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

def solve_2tsp_approx(
    tsp_matrix,
    origins,
    solution_limit=100,
    time_limit_sec=600
):
    """
    用 OR-Tools Routing API 求两旅行商 TSP 的近似解。

    参数:
      tsp_matrix     : NxN 距离矩阵（float 或 int）
      origins         : 长度为 2 的列表 [o0, o1]，表示两旅行商的起／终点节点索引
      solution_limit : 最多收集多少个可行解就停止
      time_limit_sec : 搜索总时限（秒）

    返回:
      tours          : [[o0, …, o0], [o1, …, o1]] 两条回路
      total_distance : 两条回路距离之和（同 tsp_matrix 单位）
    """
    n = len(tsp_matrix)
    # 1) 管理器 & 模型：2 辆车，从 origins[k] 出发回到 origins[k]
    manager = pywrapcp.RoutingIndexManager(n, 2, origins, origins)
    routing = pywrapcp.RoutingModel(manager)

    # 2) 距离回调（浮点×10 转整型）
    def dist_cb(from_idx, to_idx):
        i = manager.IndexToNode(from_idx)
        j = manager.IndexToNode(to_idx)
        return int(tsp_matrix[i][j] * 10)
    transit_idx = routing.RegisterTransitCallback(dist_cb)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_idx)

    # 3) 搜索参数：快速可行 + 元启发式局部搜索 + 限制
    params = pywrapcp.DefaultRoutingSearchParameters()
    params.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )      # :contentReference[oaicite:0]{index=0}
    params.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )      # :contentReference[oaicite:1]{index=1}
    params.solution_limit = solution_limit
    params.time_limit.seconds = time_limit_sec  # :contentReference[oaicite:2]{index=2}

    # 4) 求解
    sol = routing.SolveWithParameters(params)
    if not sol:
        return None, None

    # 5) 提取两条回路
    tours = []
    total_cost = 0
    for vehicle_id in range(2):
        idx = routing.Start(vehicle_id)
        tour = []
        while not routing.IsEnd(idx):
            tour.append(manager.IndexToNode(idx))
            next_idx = sol.Value(routing.NextVar(idx))
            cost = routing.GetArcCostForVehicle(idx, next_idx, vehicle_id)
            total_cost += cost
            idx = next_idx
        tour.append(manager.IndexToNode(idx))  # 回到起点
        tours.append(tour)

    # 把放大 10 倍的距离除回去
    return tours, total_cost / 10.0


# ----- 示例调用 -----
if __name__ == "__main__":
    matrix = np.random.randint(1, 100, size=(20, 20))
    np.fill_diagonal(matrix, 0)
    origins = [0, 1]  # 旅行商 A 从 0 出发回到 0，B 从 1 出发回到 1
    tours, dist = solve_2tsp_approx(matrix, origins,
                                   solution_limit=50,
                                   time_limit_sec=600)
    print("近似回路 A:", tours[0])
    print("近似回路 B:", tours[1])
    print("总距离:", dist)


近似回路 A: [0, 0]
近似回路 B: [1, 2, 16, 8, 14, 19, 15, 17, 6, 11, 4, 10, 5, 18, 12, 13, 9, 3, 7, 1]
总距离: 172.0


# 第二问

## 一体化

### 找出最远的两个点

In [22]:
max_dis = 0
max_i = 0
max_j = 0
for i in range(np.shape(data)[0]):
    for j in range(np.shape(data)[0]):
        if data[i][j] > max_dis:
            max_dis = data[i][j]
            max_i = i
            max_j = j


print(max_dis)
print(max_i)
print(max_j)

1296.6
96
79


In [11]:
from ortools.sat.python import cp_model

def solve_two_traveler_tsp_csats(
    tsp_matrix,
    origins=(0, 1),
    time_limit_sec=600,
    num_workers=20
):
    """
    一体化 CP-SAT 解两旅行商 TSP 的函数（修正 MTZ 子回路消除范围）。

    参数:
      tsp_matrix     : NxN 距离矩阵（float 或 int）
      origins         : 二元组 (o0, o1)，两旅行商的出发/回到节点索引
      time_limit_sec : 搜索总时限（秒）
      num_workers    : 并行线程数

    返回:
      tours          : [[o0,…,o0], [o1,…,o1]] 两条回路
      total_distance : 两回路距离之和（浮点数）
    """
    n = len(tsp_matrix)
    o0, o1 = origins
    model = cp_model.CpModel()

    # 1) x[k,i,j]: 第 k 位旅行者是否走 i->j
    x = {}
    for k in (0, 1):
        for i in range(n):
            for j in range(n):
                if i != j:
                    x[k, i, j] = model.NewBoolVar(f"x[{k},{i},{j}]")

    # 2) “非起点”节点恰访问一次
    for i in range(n):
        if i not in origins:
            model.Add(
                sum(x[k, j, i] for k in (0,1) for j in range(n) if j != i) == 1
            )
            model.Add(
                sum(x[k, i, j] for k in (0,1) for j in range(n) if j != i) == 1
            )

    # 3) 起点/终点约束 & 互斥访问
    for k, o in enumerate(origins):
        # 从 o 出发一次，回到 o 一次
        model.Add(sum(x[k, o, j] for j in range(n) if j != o) == 1)
        model.Add(sum(x[k, i, o] for i in range(n) if i != o) == 1)
        # 另一旅行商不得经过 o
        k2 = 1 - k
        model.Add(sum(x[k2, o, j] for j in range(n) if j != o) == 0)
        model.Add(sum(x[k2, i, o] for i in range(n) if i != o) == 0)

    # 4) 流量守恒：中间点进 = 出
    for k in (0,1):
        for i in range(n):
            if i not in origins:
                model.Add(
                    sum(x[k, j, i] for j in range(n) if j != i)
                    == sum(x[k, i, j] for j in range(n) if j != i)
                )

    # 5) MTZ 子回路消除——仅对 i,j 都不是 origin 的弧加约束
    M = n
    u = {}
    for k in (0,1):
        for i in range(n):
            if i in origins:
                u[k, i] = model.NewConstant(0)
            else:
                u[k, i] = model.NewIntVar(1, n-1, f"u[{k},{i}]")

    for k in (0,1):
        for i in range(n):
            for j in range(n):
                # 只对“中间节点间”的弧 i->j 加 MTZ
                if i not in origins and j not in origins and i != j:
                    model.Add(
                        u[k, i] + 1 <= u[k, j] + M * (1 - x[k, i, j])
                    )

    # 6) 目标：最小化总距离
    terms = []
    for k in (0,1):
        for i in range(n):
            for j in range(n):
                if i != j:
                    cost = int(tsp_matrix[i][j] * 10)
                    terms.append(cost * x[k, i, j])
    model.Minimize(sum(terms))

    # 7) 求解器配置
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = time_limit_sec
    solver.parameters.num_search_workers    = num_workers

    status = solver.Solve(model)
    if status not in (cp_model.OPTIMAL, cp_model.FEASIBLE):
        return None, None

    # 8) 提取回路
    tours = []
    for k, o in enumerate(origins):
        path = [o]
        curr = o
        while True:
            nxt = None
            for j in range(n):
                if j != curr and solver.Value(x[k, curr, j]) == 1:
                    nxt = j
                    break
            if nxt is None or nxt == o:
                break
            path.append(nxt)
            curr = nxt
        path.append(o)
        tours.append(path)

    total_dist = solver.ObjectiveValue() / 10
    return tours, total_dist

import numpy as np
import time
from ortools.sat.python import cp_model

def test_solve_two_traveler_tsp_csats():
    # 生成一个20x20的随机距离矩阵
    # np.random.seed(42)  # 设置随机种子以便结果可复现
    n = 20
    matrix = np.random.randint(500, 1000, size=(n, n)).astype(float)
    # matrix = data
    # 确保对角线为0（自己到自己的距离为0）
    np.fill_diagonal(matrix, 0)

    # # 确保矩阵是对称的（可选，取决于您的问题设置）
    # for i in range(n):
    #     for j in range(i+1, n):
    #         matrix[j, i] = matrix[i, j]

    # 设置起点
    origins = (1, 2)  # 第一个旅行者从0出发，第二个从1出发

    print(f"测试矩阵大小: {n}x{n}")
    print(f"起点设置: {origins}")

    # 记录开始时间
    start_time = time.time()

    # 调用求解函数
    tours, total_distance = solve_two_traveler_tsp_csats(
        matrix,
        origins=origins,
        time_limit_sec=120,  # 2分钟时间限制
        num_workers=20        # 4个并行线程
    )

    # 计算耗时
    elapsed_time = time.time() - start_time

    # 输出结果
    if tours is not None:
        print("\n求解成功!")
        print(f"求解耗时: {elapsed_time:.2f} 秒")
        print(f"总距离: {total_distance:.2f}")
        print(f"旅行者1路径: {tours[0]}")
        print(f"旅行者2路径: {tours[1]}")

        # 验证每个节点是否被恰好访问一次
        all_nodes = set(range(n))
        visited = set()
        for tour in tours:
            # 去掉每个路径中的最后一个节点（回到起点）
            for node in tour[:-1]:
                visited.add(node)

        missed = all_nodes - visited
        if missed:
            print(f"警告: 有节点未被访问: {missed}")
        else:
            print("验证成功: 所有节点均被访问一次")
    else:
        print("\n未找到可行解")
        print(f"耗时: {elapsed_time:.2f} 秒")

if __name__ == "__main__":
    test_solve_two_traveler_tsp_csats()


测试矩阵大小: 20x20
起点设置: (1, 2)

求解成功!
求解耗时: 0.22 秒
总距离: 10641.00
旅行者1路径: [1, 9, 12, 0, 10, 19, 13, 5, 4, 11, 17, 1]
旅行者2路径: [2, 6, 7, 18, 15, 8, 16, 3, 14, 2]
验证成功: 所有节点均被访问一次


### 真实数据

In [12]:
n = 107
origins = (96, 79)
# print(f"测试矩阵大小: {n}x{n}")
print(f"起点设置: {origins}")
# 记录开始时间
start_time = time.time()

# 调用求解函数
tours, total_distance = solve_two_traveler_tsp_csats(
    data,
    origins=origins,
    time_limit_sec=1200,  # 10分钟时间限制
    num_workers=24        # 20个并行线程
)

elapsed_time = time.time() - start_time

if tours is not None:
    print("\n求解成功!")
    print(f"求解耗时: {elapsed_time:.2f} 秒")
    print(f"总距离: {total_distance:.2f}")
    print(f"旅行者1路径: {tours[0]}")
    print(f"旅行者2路径: {tours[1]}")

    # 验证每个节点是否被恰好访问一次
    all_nodes = set(range(n))
    visited = set()
    for tour in tours:
        # 去掉每个路径中的最后一个节点（回到起点）
        for node in tour[:-1]:
            visited.add(node)

    missed = all_nodes - visited
    if missed:
        print(f"警告: 有节点未被访问: {missed}")
    else:
        print("验证成功: 所有节点均被访问一次")
else:
    print("\n未找到可行解")
    print(f"耗时: {elapsed_time:.2f} 秒")

起点设置: (96, 79)

求解成功!
求解耗时: 1204.48 秒
总距离: 6172.10
旅行者1路径: [96, 94, 89, 90, 93, 74, 70, 91, 92, 76, 69, 68, 66, 67, 72, 73, 71, 75, 27, 18, 17, 24, 25, 20, 19, 28, 23, 21, 26, 22, 40, 10, 11, 30, 29, 41, 106, 34, 33, 35, 36, 37, 38, 39, 32, 31, 12, 6, 50, 15, 13, 14, 16, 65, 62, 61, 60, 53, 57, 58, 59, 82, 81, 80, 77, 85, 84, 88, 86, 83, 56, 87, 55, 54, 63, 64, 51, 46, 47, 49, 48, 42, 43, 45, 52, 44, 100, 99, 102, 101, 103, 9, 7, 3, 4, 2, 0, 1, 5, 8, 105, 104, 97, 98, 95, 96]
旅行者2路径: [79, 78, 79]
验证成功: 所有节点均被访问一次


## 聚类

In [45]:
import numpy as np
from sklearn.cluster import AgglomerativeClustering

def split_into_two_subgraphs(data):
    """
    将给定的距离矩阵 data 划分成两簇子图，返回各自的节点索引和子距离矩阵。

    参数:
      data : (N, N) 距离矩阵（NumPy 数组或等价类型），需满足 data[i,i]=0 且对称

    返回:
      idx_a, idx_b : 分到子图 A、B 的原始节点索引列表
      data_a, data_b : 分别对应的子距离矩阵，形状 (len(idx_a), len(idx_a)) 和 (len(idx_b), len(idx_b))
    """
    # 确保是 NumPy 数组
    data = np.array(data)
    n = data.shape[0]
    if data.shape[1] != n:
        raise ValueError("输入矩阵必须是方阵")

    # 1) 用层次聚类（预计算距离）划分成两簇
    model = AgglomerativeClustering(
        n_clusters=2,
        metric="precomputed",   # 直接使用距离矩阵
        linkage="average"       # 平均距离合并
    )
    labels = model.fit_predict(data)

    # 2) 提取每簇的节点索引
    idx_a = np.where(labels == 0)[0].tolist()
    idx_b = np.where(labels == 1)[0].tolist()

    # 3) 构造子距离矩阵
    data_a = data[np.ix_(idx_a, idx_a)]
    data_b = data[np.ix_(idx_b, idx_b)]

    return idx_a, idx_b, data_a, data_b

# --------- 示例使用 ----------
if __name__ == "__main__":
    # 假设有一个 5x5 距离矩阵 data
    data = [
        [0.0, 2.1, 3.5, 4.0, 1.8],
        [2.1, 0.0, 2.0, 3.2, 2.5],
        [3.5, 2.0, 0.0, 1.7, 3.1],
        [4.0, 3.2, 1.7, 0.0, 2.2],
        [1.8, 2.5, 3.1, 2.2, 0.0],
    ]
    idx_a, idx_b, data_a, data_b = split_into_two_subgraphs(data)
    print("子图 A 节点索引：", idx_a)
    print("子图 B 节点索引：", idx_b)
    print("子图 A 距离矩阵：\n", data_a)
    print("子图 B 距离矩阵：\n", data_b)


子图 A 节点索引： [0, 1, 4]
子图 B 节点索引： [2, 3]
子图 A 距离矩阵：
 [[0.  2.1 1.8]
 [2.1 0.  2.5]
 [1.8 2.5 0. ]]
子图 B 距离矩阵：
 [[0.  1.7]
 [1.7 0. ]]


In [50]:
idx_a, idx_b, data_a, data_b = split_into_two_subgraphs(data)
print(len(idx_a))
# print(idx_b)

86


## 分别使用

In [56]:
tour_a, total_distance_a = solve_tsp_ortools_new(data_a)

if tour:
    print("Optimal tour:", tour_a)
    print("Total distance:", total_distance_a)
else:
    print("No solution found.")

Optimal tour: [0, 1, 5, 8, 84, 83, 76, 77, 74, 75, 73, 68, 69, 72, 65, 61, 70, 71, 67, 60, 59, 57, 58, 63, 64, 62, 66, 27, 18, 17, 24, 25, 20, 19, 28, 23, 21, 26, 22, 40, 10, 11, 30, 29, 41, 85, 34, 33, 35, 36, 37, 38, 39, 32, 31, 12, 6, 50, 15, 13, 14, 16, 56, 53, 54, 55, 51, 46, 47, 49, 48, 42, 43, 45, 52, 44, 79, 78, 81, 80, 82, 9, 7, 3, 4, 2, 0]
Total distance: 4724.1


In [57]:
tour_b, total_distance_b = solve_tsp_ortools_new(data_b)

if tour:
    print("Optimal tour:", tour_b)
    print("Total distance:", total_distance_b)
else:
    print("No solution found.")

Optimal tour: [0, 1, 2, 19, 3, 15, 18, 20, 16, 17, 9, 10, 11, 12, 13, 14, 6, 5, 4, 7, 8, 0]
Total distance: 1773.8


In [59]:
tour_a_real = [idx_a[i] for i in tour_a]
tour_b_real = [idx_b[i] for i in tour_b]
print(tour_a_real)
print(tour_b_real)
print(total_distance_a + total_distance_b)

[0, 1, 5, 8, 105, 104, 97, 98, 95, 96, 94, 89, 90, 93, 74, 70, 91, 92, 76, 69, 68, 66, 67, 72, 73, 71, 75, 27, 18, 17, 24, 25, 20, 19, 28, 23, 21, 26, 22, 40, 10, 11, 30, 29, 41, 106, 34, 33, 35, 36, 37, 38, 39, 32, 31, 12, 6, 50, 15, 13, 14, 16, 65, 62, 63, 64, 51, 46, 47, 49, 48, 42, 43, 45, 52, 44, 100, 99, 102, 101, 103, 9, 7, 3, 4, 2, 0]
[53, 54, 55, 87, 56, 83, 86, 88, 84, 85, 77, 78, 79, 80, 81, 82, 59, 58, 57, 60, 61, 53]
6497.900000000001


## 使用裁剪的方法

In [42]:
one_tr_tour = [0, 1, 5, 8, 105, 104, 97, 98, 95, 96, 94, 89, 90, 93, 74, 70, 91, 92, 76, 69, 68, 66, 67, 72, 73, 71, 75, 27, 18, 17, 24, 25, 20, 19, 28, 23, 21, 26, 22, 40, 10, 11, 30, 29, 41, 106, 34, 33, 35, 36, 37, 38, 39, 32, 31, 12, 6, 50, 15, 13, 14, 16, 65, 62, 61, 60, 53, 57, 58, 59, 82, 81, 80, 78, 79, 77, 85, 84, 88, 86, 83, 56, 87, 55, 54, 63, 64, 51, 46, 47, 49, 48, 42, 43, 45, 52, 44, 100, 99, 102, 101, 103, 9, 7, 3, 4, 2]

extra_cost = []
max_cost = 0
best_split = None
for i in range(107):
    for j in range(107):
        id1 = i
        id2 = (i + 1) % 107
        id3 = j
        id4 = (j + 1) % 107
        if id1 == id3 or id1 == id4 or id2 == id3 or id2 == id4:
            continue
        cost = data[one_tr_tour[id1]][one_tr_tour[id2]] + data[one_tr_tour[id3]][one_tr_tour[id4]] - data[one_tr_tour[id1]][one_tr_tour[id4]] - data[one_tr_tour[id3]][one_tr_tour[id2]]
        extra_cost.append(cost)
        if cost > max_cost:
            max_cost = cost
            best_split = (one_tr_tour[id1], one_tr_tour[id2], one_tr_tour[id3], one_tr_tour[id4])

extra_cost = sorted(extra_cost)
print(extra_cost[0], extra_cost[-1])
print(best_split)

-2196.9 210.59999999999997
(80, 78, 79, 77)


## 便宜

In [16]:
def solve_tsp_insertion(tsp_matrix):
    """
    使用“最便宜插入”（Cheapest Insertion）启发式算法求解满足三角不等式的正权 TSP。

    参数:
        tsp_matrix: NxN 的距离矩阵（正实数或整数），满足 d[i][j] = d[j][i], d[i][i] = 0

    返回:
        tour: 按访问顺序列出的节点索引列表（首尾相同），如 [0, 2, 5, 1, 0]
        total_distance: 对应回路的总距离
    """
    n = len(tsp_matrix)
    if n == 0:
        return [], 0
    if n == 1:
        return [0, 0], 0

    # 初始回路：从节点 0 到节点 1 回到 0
    tour = [3, 4, 3]
    remaining = set(range(n)) - {0, 1}

    # 不断插入最便宜的节点
    while remaining:
        best_node = None
        best_pos = None
        best_increase = float('inf')

        # 遍历每个待插入节点
        for node in remaining:
            # 尝试插入到 tour 的每一条边 (i -> j) 之间
            for i in range(len(tour) - 1):
                u, v = tour[i], tour[i+1]
                increase = (
                    tsp_matrix[u][node]
                    + tsp_matrix[node][v]
                    - tsp_matrix[u][v]
                )
                if increase < best_increase:
                    best_increase = increase
                    best_node = node
                    best_pos = i + 1

        # 在最佳位置插入该节点
        tour.insert(best_pos, best_node)
        remaining.remove(best_node)

    # 计算总距离
    total_distance = sum(
        tsp_matrix[tour[i]][tour[i+1]] for i in range(len(tour) - 1)
    )

    return tour, total_distance


# 示例调用
if __name__ == "__main__":
    matrix = [
        [0,   2.1, 3.5, 4.0, 1.8],
        [2.1, 0,   2.0, 3.2, 2.5],
        [3.5, 2.0, 0,   1.7, 3.1],
        [4.0, 3.2, 1.7, 0,   2.2],
        [1.8, 2.5, 3.1, 2.2, 0  ],
    ]
    tour, dist = solve_tsp_insertion(matrix)
    print("插入法回路：", tour)
    print("总距离：", dist)


插入法回路： [3, 3, 2, 4, 4, 3]
总距离： 7.0


In [17]:
tour, dist = solve_tsp_insertion(data)
print("插入法回路：", tour)
print("总距离：", dist)

插入法回路： [3, 3, 5, 105, 104, 8, 2, 4, 4, 29, 41, 106, 11, 10, 40, 23, 21, 20, 19, 28, 67, 66, 69, 70, 74, 90, 93, 94, 89, 96, 95, 98, 97, 91, 92, 76, 68, 72, 73, 71, 75, 27, 17, 18, 24, 25, 22, 26, 38, 36, 37, 35, 33, 34, 30, 32, 39, 31, 12, 64, 63, 62, 53, 82, 59, 58, 57, 56, 86, 85, 77, 78, 79, 81, 80, 84, 88, 83, 87, 55, 54, 60, 61, 65, 16, 14, 13, 15, 50, 6, 48, 49, 47, 46, 51, 45, 52, 44, 99, 102, 101, 103, 100, 43, 42, 7, 9, 3]
总距离： 7827.499999999996


In [26]:
def solve_tsp_insertion(tsp_matrix):
    """
    使用"最便宜插入"（Cheapest Insertion）启发式算法求解TSP，
    然后用3-opt局部搜索优化解。

    参数:
        tsp_matrix: NxN 的距离矩阵（正实数或整数），满足 d[i][j] = d[j][i], d[i][i] = 0

    返回:
        tour: 按访问顺序列出的节点索引列表（首尾相同），如 [0, 2, 5, 1, 0]
        total_distance: 对应回路的总距离
    """
    n = len(tsp_matrix)
    if n <= 1:
        return [0, 0], 0

    # 初始回路：从节点 0 到节点 1 回到 0
    tour = [0, 1, 0]
    remaining = set(range(n)) - {0, 1}

    # 不断插入最便宜的节点
    while remaining:
        best_node = None
        best_pos = None
        best_increase = float('inf')

        # 遍历每个待插入节点
        for node in remaining:
            # 尝试插入到 tour 的每一条边 (i -> j) 之间
            for i in range(len(tour) - 1):
                u, v = tour[i], tour[i+1]
                increase = (
                    tsp_matrix[u][node]
                    + tsp_matrix[node][v]
                    - tsp_matrix[u][v]
                )
                if increase < best_increase:
                    best_increase = increase
                    best_node = node
                    best_pos = i + 1

        # 在最佳位置插入该节点
        tour.insert(best_pos, best_node)
        remaining.remove(best_node)

    # 计算初始总距离
    initial_distance = sum(tsp_matrix[tour[i]][tour[i+1]] for i in range(len(tour) - 1))

    # 应用3-opt优化
    tour = three_opt(tour[:-1], tsp_matrix)  # 移除重复的起点，优化后再添加
    tour.append(tour[0])  # 添加回起点，形成完整回路

    # 计算优化后总距离
    total_distance = sum(tsp_matrix[tour[i]][tour[i+1]] for i in range(len(tour) - 1))

    return tour, total_distance

import numpy as np
from joblib import Parallel, delayed
import time

def three_opt(tour, dist_matrix, n_jobs=-1, chunk_size=50, max_iterations=3):
    """
    使用3-opt局部搜索优化TSP路径，通过并行计算提高效率

    参数:
        tour: 初始路径（不包含重复的起点）
        dist_matrix: 距离矩阵
        n_jobs: 并行作业数量，默认为-1（使用所有可用CPU核心）
        chunk_size: 每批并行处理的边组合数量
        max_iterations: 最大迭代次数，避免陷入过长的搜索

    返回:
        优化后的路径和总距离
    """
    n = len(tour)
    best_tour = tour.copy()
    improved = True
    iteration = 0

    # 预计算距离查找表以加速计算
    dist_lookup = {}
    for i in range(n):
        for j in range(n):
            dist_lookup[(tour[i], tour[j])] = dist_matrix[tour[i]][tour[j]]

    def evaluate_swap(i, j, k):
        """评估特定的3-opt交换操作"""
        a, b = best_tour[i], best_tour[(i + 1) % n]
        c, d = best_tour[j], best_tour[(j + 1) % n]
        e, f = best_tour[k], best_tour[(k + 1) % n]

        # 当前配置距离
        d0 = dist_lookup[(a, b)] + dist_lookup[(c, d)] + dist_lookup[(e, f)]

        # 三种可能的重连方式
        d1 = dist_lookup[(a, c)] + dist_lookup[(b, e)] + dist_lookup[(d, f)]  # 方式1
        d2 = dist_lookup[(a, d)] + dist_lookup[(c, f)] + dist_lookup[(e, b)]  # 方式2
        d3 = dist_lookup[(a, e)] + dist_lookup[(d, b)] + dist_lookup[(c, f)]  # 方式3

        improvements = []
        if d1 < d0:
            improvements.append((d0 - d1, 1, i, j, k))
        if d2 < d0:
            improvements.append((d0 - d2, 2, i, j, k))
        if d3 < d0:
            improvements.append((d0 - d3, 3, i, j, k))

        return improvements if improvements else []

    def generate_swap_indices():
        """生成要评估的所有索引组合"""
        indices = []
        for i in range(n - 2):
            for j in range(i + 1, n - 1):
                for k in range(j + 1, n):
                    indices.append((i, j, k))
        return indices

    start_time = time.time()
    print("开始3-opt优化...")

    while improved and iteration < max_iterations:
        improved = False
        iteration += 1

        # 生成所有可能的3-opt交换索引
        all_indices = generate_swap_indices()

        # 使用joblib进行并行处理
        results = Parallel(n_jobs=n_jobs, prefer="threads")(
            delayed(evaluate_swap)(i, j, k)
            for i, j, k in all_indices
        )

        # 扁平化结果列表
        all_improvements = []
        for r in results:
            all_improvements.extend(r)

        # 如果找到改进，选择最佳改进
        if all_improvements:
            # 按改进量排序，选择最大的
            best_improvement = max(all_improvements, key=lambda x: x[0])
            improvement, move_type, i, j, k = best_improvement

            # 执行最佳交换
            if move_type == 1:
                # 翻转从i+1到j
                best_tour[i+1:j+1] = best_tour[i+1:j+1][::-1]
            elif move_type == 2:
                # 翻转从j+1到k
                best_tour[j+1:k+1] = best_tour[j+1:k+1][::-1]
            elif move_type == 3:
                # 交换i+1到j和j+1到k的部分
                temp = best_tour[i+1:j+1].copy()
                best_tour[i+1:i+1+(k-j)] = best_tour[j+1:k+1]
                best_tour[i+1+(k-j):k+1] = temp

            # 更新查找表（实际操作中可能需要根据修改更新部分查找表项）
            for ii in range(n):
                for jj in range(n):
                    dist_lookup[(best_tour[ii], best_tour[jj])] = dist_matrix[best_tour[ii]][best_tour[jj]]

            improved = True

        elapsed = time.time() - start_time
        print(f"迭代 {iteration}: 找到改进 = {improved}, 已用时间 = {elapsed:.2f}秒")

    # 计算最终路径的总距离
    total_distance = 0
    for i in range(len(best_tour)):
        j = (i + 1) % len(best_tour)
        total_distance += dist_matrix[best_tour[i]][best_tour[j]]

    elapsed_time = time.time() - start_time
    print(f"3-opt优化完成，共进行了{iteration}次迭代，总用时{elapsed_time:.2f}秒")

    return best_tour, total_distance

def solve_tsp_insertion(tsp_matrix):
    """
    使用"最便宜插入"（Cheapest Insertion）启发式算法求解TSP，
    然后用3-opt局部搜索优化解。
    """
    n = len(tsp_matrix)
    if n <= 1:
        return [0, 0], 0

    # 初始回路：从节点 0 到节点 1 回到 0
    tour = [0, 1, 0]
    remaining = set(range(n)) - {0, 1}

    # 不断插入最便宜的节点
    while remaining:
        best_node = None
        best_pos = None
        best_increase = float('inf')

        # 遍历每个待插入节点
        for node in remaining:
            # 尝试插入到 tour 的每一条边 (i -> j) 之间
            for i in range(len(tour) - 1):
                u, v = tour[i], tour[i+1]
                increase = (
                    tsp_matrix[u][node]
                    + tsp_matrix[node][v]
                    - tsp_matrix[u][v]
                )
                if increase < best_increase:
                    best_increase = increase
                    best_node = node
                    best_pos = i + 1

        # 在最佳位置插入该节点
        tour.insert(best_pos, best_node)
        remaining.remove(best_node)

    # 计算初始总距离
    initial_distance = sum(tsp_matrix[tour[i]][tour[i+1]] for i in range(len(tour) - 1))

    # 应用3-opt优化 - 正确解析返回值
    optimized_tour, optimized_distance = three_opt(
        tour=tour[:-1],  # 移除重复的起点
        dist_matrix=tsp_matrix,
        n_jobs=20,
        max_iterations=5
    )

    # 添加回起点，形成完整回路
    complete_tour = optimized_tour.copy()
    complete_tour.append(complete_tour[0])

    return complete_tour, optimized_distance

In [27]:
tour, dist = solve_tsp_insertion(data)
# optimized_tour, optimized_distance = three_opt(
#     tour=tour[:-1],  # 移除末尾重复的起点
#     dist_matrix=data,
#     n_jobs=20,  # 使用4个核心并行计算
#     max_iterations=5  # 最多迭代5次
# )

# 添加回起点，形成完整回路
# optimized_tour.append(optimized_tour[0])
#
# print(f"3-opt优化后的路径长度: {len(optimized_tour)}")
# print(f"3-opt优化后的总距离: {optimized_distance:.2f}")

print("插入法回路：", tour)
print("总距离：", dist)

开始3-opt优化...
迭代 1: 找到改进 = True, 已用时间 = 3.85秒
迭代 2: 找到改进 = True, 已用时间 = 7.66秒
迭代 3: 找到改进 = True, 已用时间 = 11.39秒
迭代 4: 找到改进 = True, 已用时间 = 15.20秒
迭代 5: 找到改进 = True, 已用时间 = 19.17秒
3-opt优化完成，共进行了5次迭代，总用时19.17秒
插入法回路： [0, 1, 105, 104, 8, 5, 4, 2, 29, 41, 106, 11, 10, 40, 23, 21, 20, 19, 28, 67, 66, 69, 70, 74, 93, 90, 94, 89, 96, 95, 98, 97, 91, 92, 76, 68, 72, 73, 71, 75, 27, 17, 18, 24, 25, 22, 26, 38, 36, 37, 35, 33, 34, 30, 32, 39, 31, 12, 6, 50, 15, 13, 14, 16, 65, 61, 60, 54, 55, 87, 83, 88, 84, 78, 77, 85, 86, 79, 80, 81, 82, 59, 58, 57, 56, 53, 62, 63, 64, 48, 49, 47, 46, 51, 45, 52, 44, 99, 102, 101, 103, 100, 43, 42, 7, 9, 3, 0]
总距离： 7724.299999999998


## 拉格朗日松弛

In [36]:
def solve_two_traveler_tsp_csats_lower_bound(
    tsp_matrix,
    origins=(96, 79),
    time_limit_sec=600,
    num_workers=20
):
    """
    计算两旅行商 TSP 的下界（assignment relaxation）。

    参数:
      tsp_matrix     : NxN 距离矩阵（float 或 int）
      origins         : 二元组 (o0, o1)，两旅行商的起点（对下界无影响，可传入）
      time_limit_sec : 搜索时限（对下界计算无影响，可传入）
      num_workers    : 并行线程数（对下界计算无影响，可传入）

    返回:
      lower_bound    : 原整数规划最优值的一个下界（浮点数）
    """
    n = len(tsp_matrix)
    if n == 0:
        return 0.0

    # 对每个节点 i，我们松弛“连通性/Subtour”约束，
    # 只保留“度数 = 2”约束，那么对每个 i 最低代价
    # 就是选它相连的两条最短边。
    total = 0.0
    for i in range(n):
        # 排除 i→i，找两条最小的 d[i][j]
        # O(n log n) 排序亦可优化为 O(n)
        dists = sorted(tsp_matrix[i][j] for j in range(n) if j != i)
        total += dists[0] + dists[1]

    # 因为每条边在 sum_i(deg 2) 时被统计了两次，所以除以 2
    lower_bound = total * 0.5
    return lower_bound


In [37]:
# # 使用示例
# best_lb, best_tours = lagrangian_2tsp_lower_bound(
#     tsp_matrix=data,
#     origins=(96, 79),
#     max_iter=200,
#     step_size=0.05
# )
#
# print(f"两旅行商问题的拉格朗日对偶下界: {best_lb:.2f}")
# print(f"旅行者1路径: {best_tours[0]}")
# print(f"旅行者2路径: {best_tours[1]}")
lb = solve_two_traveler_tsp_csats_lower_bound(data)
print(f"拉格朗日下界: {lb:.2f}")

拉格朗日下界: 5217.45
